# _Updates to make -- from Carolyn's notes, 082018:_

City-suburb housing production

----

Author: Dana Chermesh, Regional Planning intern;
July 2018

### _US Metros comparison  Notebook no.6_ 
# Housing Permitted 2010-2017
### retrieved from _Census Bureau Building Permits Survey County and Place level for 2017 annually_

----


In [2]:
import pandas as pd

# reading in my api key saved in censusAPI.py as
# myAPI = 'XXXXXXXXXXXXXXX'
# request an api key in: https://api.census.gov/data/key_signup.html
from censusAPI import myAPI

# Data

## 1.1 Total housing units 2000 (base point); _Decennial Census 2000, SF1_
Data were retrieved using my census API (_**see explanation in README.md of this repo or in former notebooks**_)
- [variables](https://api.census.gov/data/2000/sf1/variables.html)

In [3]:
# total POP and total housing units for all counties in the US, 2010
# P001001 = total pop
# H001001 = total housing units

housing00 = pd.read_json('https://api.census.gov/data/2000/sf1?get=P001001,H001001'+
                         '&for=county:*&in=state:*&key='+myAPI)
housing00.columns = housing00.iloc[0]
housing00 = housing00[1:]

housing00.columns = ['pop2000', 'hu2000', 'state', 'county']
housing00['STCO'] = housing00[['state', 'county']].apply(lambda x: ''.join(x), axis=1)

print(housing00.shape)
housing00.head()

(3141, 5)


,pop2000,hu2000,state,county,STCO
1,43671,17662,01,001,01001
2,140415,74285,01,003,01003
3,29038,12461,01,005,01005
4,20826,8345,01,007,01007
5,51024,21158,01,009,01009


## Total housing units 2010 (base point); _Decennial Census 2010, SF1_
Data were retrieved using my census API (_**see explanation in README.md of this repo or in former notebooks**_)
- [variables](https://api.census.gov/data/2010/sf1/variables.html)

In [4]:
# total POP and total housing units for all counties in the US, 2010
# P0010001 = total pop
# H00010001 = total housing units

housing10 = pd.read_json('https://api.census.gov/data/2010/sf1?get=P0010001,H00010001'+
                         '&for=county:*&in=state:*&key='+myAPI)
housing10.columns = housing10.iloc[0]
housing10 = housing10[1:]

housing10.columns = ['pop2010', 'hu2010', 'state', 'county']
housing10['STCO'] = housing10[['state', 'county']].apply(lambda x: ''.join(x), axis=1)

print(housing10.shape)
housing10.head()

(3221, 5)


,pop2010,hu2010,state,county,STCO
1,54571,22135,01,001,01001
2,182265,104061,01,003,01003
3,27457,11829,01,005,01005
4,22915,8981,01,007,01007
5,57322,23887,01,009,01009


# 1.2 Perimts issueance data from 2010 to 2017 - [Building Permits Survey](https://www.census.gov/construction/bps/)
The data were retrieved from the Census Bureau [Building Permits Survey](https://www.census.gov/construction/bps/), [Permits by County or Place](http://www2.census.gov/econ/bps).

For downloading the data, please go to the [County/](https://www2.census.gov/econ/bps/County/) page or the [Places/](https://www2.census.gov/econ/bps/Place/) page, and choose:
- [co2017a.txt](https://www2.census.gov/econ/bps/County/co2017a.txt) 
- [co2016a.txt](https://www2.census.gov/econ/bps/County/co2016a.txt) 
- [co2015a.txt](https://www2.census.gov/econ/bps/County/co2015a.txt) 
- [co2014a.txt](https://www2.census.gov/econ/bps/County/co2014a.txt) 
- [co2013a.txt](https://www2.census.gov/econ/bps/County/co2013a.txt) 
- [co2012a.txt](https://www2.census.gov/econ/bps/County/co2012a.txt) 
- [co2011a.txt](https://www2.census.gov/econ/bps/County/co2011a.txt) 
- [co2010a.txt](https://www2.census.gov/econ/bps/County/co2010a.txt) 

and:
- [ne2017a.txt](https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2017a.txt)
- [mw2017a.txt](https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2017a.txt)
- [so2017a.txt](https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2017a.txt)
- [we2017a.txt](https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2017a.txt)<br>
.<br>
.<br>
.<br>
.<br>

Data can be read directly to this notebook using pandas `read_table`, as bellow.

## 2010-2017 Permits 

In [21]:
tablesCO = ['co2017a', 'co2016a', 'co2015a', 'co2014a',
            'co2013a', 'co2012a', 'co2011a', 'co2010a']

COdata10 = []

for year in tablesCO:
    
    df = pd.read_table('https://www2.census.gov/econ/bps/County/'+year+'.txt', 
            header=0, sep=r'\,|\t', engine='python').iloc[:,:18]

    df.columns = df.iloc[0]
    df = df[1:].set_index(['Name'])

    df = df.drop(['Code','Bldgs', 'Value'], axis=1)
    df.columns = ['State', 'County', '1unit', '2unit', '3-4unit', '+5unit']

    df = df.astype(int)
    df['1-2units'] = df['1unit'] + df['2unit']
    df['+3units'] = df['3-4unit'] + df['+5unit']
    df = df.drop(['1unit', '2unit', '3-4unit', '+5unit'], axis=1)

    df['State'] = df['State'].apply(lambda x: '{0:0>2}'.format(x))
    df['County'] = df['County'].apply(lambda x: '{0:0>3}'.format(x))

    df['STCO'] = df[['State', 'County']].apply(lambda x: ''.join(x), axis=1)
    print('data:',year ,df.shape)
    COdata10.append(df)

COall10_17 = pd.concat(COdata10).groupby('STCO').sum()
print(COall10_17.shape)
print(COall10_17.dtypes)
COall10_17.head()

data: co2017a (3038, 5)
data: co2016a (3039, 5)
data: co2015a (3044, 5)
data: co2014a (3038, 5)
data: co2013a (3027, 5)
data: co2012a (3026, 5)
data: co2011a (3026, 5)
data: co2010a (3026, 5)
(3061, 2)
1-2units    int64
+3units     int64
dtype: object


,1-2units,+3units
STCO,,
01001,1136,312
01003,11319,1168
01005,56,40
01007,102,0
01009,74,40


## 2000-2010 Permits

In [23]:
tablesCO00 = ['co2009a', 'co2008a', 'co2007a', 'co2006a',
              'co2005a', 'co2004a', 'co2003a', 'co2002a',
              'co2001a', 'co2000a']

COdata00 = []

for year in tablesCO00:

    df = pd.read_table('https://www2.census.gov/econ/bps/County/'+year+'.txt', 
            header=0, sep=r'\,|\t', engine='python').iloc[:,:18]

    df.columns = df.iloc[0]
    df = df[1:].set_index(['Name'])

    df = df.drop(['Code','Bldgs', 'Value'], axis=1)
    df.columns = ['State', 'County', '1unit', '2unit', '3-4unit', '+5unit']

    df = df.astype(int)
    df['1-2units_00'] = df['1unit'] + df['2unit']
    df['+3units_00'] = df['3-4unit'] + df['+5unit']
    df = df.drop(['1unit', '2unit', '3-4unit', '+5unit'], axis=1)

    df['State'] = df['State'].apply(lambda x: '{0:0>2}'.format(x))
    df['County'] = df['County'].apply(lambda x: '{0:0>3}'.format(x))

    df['STCO'] = df[['State', 'County']].apply(lambda x: ''.join(x), axis=1)
    print('data:',year ,df.shape)
    COdata00.append(df)

COall00_10 = pd.concat(COdata00).groupby('STCO').sum()
print(COall00_10.shape)
print(COall00_10.dtypes)
COall00_10.head()

data: co2009a (3026, 5)
data: co2008a (3026, 5)
data: co2007a (3026, 5)
data: co2006a (3026, 5)
data: co2005a (3027, 5)
data: co2004a (3027, 5)
data: co2003a (3021, 5)
data: co2002a (3013, 5)
data: co2001a (3014, 5)
data: co2000a (3018, 5)
(3050, 2)
1-2units_00    int64
+3units_00     int64
dtype: object


,1-2units_00,+3units_00
STCO,,
01001,2486,224
01003,23170,10093
01005,205,40
01007,80,42
01009,422,91


## 1.3 Reading in geo-coded dataset
created on a different notebook, please refer to _**ADD NOTEBOOK NAME**_

In [24]:
geo = pd.read_csv('data/USmetros_full.csv').iloc[:,:-2] \
            .drop(['Unnamed: 0', 'SHAPE_AREA'], axis=1)
geo['STCO'] = geo['STCO'].apply(lambda x: '{0:0>5}'.format(x))

print(geo.shape)
geo.head()

(270, 4)


,CSA,CSA_name,County_name,STCO
0,488,"San Jose-San Francisco-Oakland, CA",Alameda,06001
1,488,"San Jose-San Francisco-Oakland, CA",Contra Costa,06013
2,488,"San Jose-San Francisco-Oakland, CA",Marin,06041
3,488,"San Jose-San Francisco-Oakland, CA",Napa,06055
4,488,"San Jose-San Francisco-Oakland, CA",San Benito,06069


## 1.4 Merging datasets

### merging 00-10 with 10-17 datasets

In [25]:
permits = COall00_10.merge(COall10_17, on='STCO')

print(permits.shape)
permits.head()

(3029, 4)


,1-2units_00,+3units_00,1-2units,+3units
STCO,,,,
01001,2486,224,1136,312
01003,23170,10093,11319,1168
01005,205,40,56,40
01007,80,42,102,0
01009,422,91,74,40


### merging permits with geo dataset

In [26]:
BPcounties = permits.merge(geo, left_index=True, right_on='STCO').set_index('County_name')

print(BPcounties.shape)
BPcounties.head()

(270, 7)


,1-2units_00,+3units_00,1-2units,+3units,CSA,CSA_name,STCO
County_name,,,,,,,
Alameda,18133,19628,13298,19916,488,"San Jose-San Francisco-Oakland, CA",06001
Contra Costa,36575,9723,11794,4309,488,"San Jose-San Francisco-Oakland, CA",06013
Los Angeles,87859,96181,35396,90987,348,"Los Angeles-Long Beach, CA",06037
Marin,3690,1806,1103,1004,488,"San Jose-San Francisco-Oakland, CA",06041
Napa,4711,1154,1027,278,488,"San Jose-San Francisco-Oakland, CA",06055


In [27]:
print(BPcounties[BPcounties['CSA']==408].shape)
BPcounties[BPcounties['CSA']==408]

(31, 7)


,1-2units_00,+3units_00,1-2units,+3units,CSA,CSA_name,STCO
County_name,,,,,,,
Fairfield,14812,6385,6127,8484,408,"New York-Newark, NY-NJ-CT-PA",09001
Litchfield,5754,111,1045,98,408,"New York-Newark, NY-NJ-CT-PA",09005
New Haven,12615,3540,3715,3446,408,"New York-Newark, NY-NJ-CT-PA",09009
Bergen,11629,8434,7683,12706,408,"New York-Newark, NY-NJ-CT-PA",34003
Essex,10600,8962,2987,11582,408,"New York-Newark, NY-NJ-CT-PA",34013
Hudson,6313,20300,2249,25364,408,"New York-Newark, NY-NJ-CT-PA",34017
Hunterdon,4668,343,1185,1169,408,"New York-Newark, NY-NJ-CT-PA",34019
Mercer,8158,2474,1580,3487,408,"New York-Newark, NY-NJ-CT-PA",34021
Middlesex,13926,6753,6561,9163,408,"New York-Newark, NY-NJ-CT-PA",34023


### Exporting all counties housing permits 2017 data to .csv

In [28]:
BPcounties.to_csv('exports/BPcounties00-17.csv')

## groupby CSAs to sum housing permits by Metro, merging 2010 Census Data of total housing units and total pop 

In [29]:
housing00['pop2000'] = housing00['pop2000'].astype(int)
housing00['hu2000'] = housing00['hu2000'].astype(int)

housing00.dtypes

pop2000     int64
hu2000      int64
state      object
county     object
STCO       object
dtype: object

In [30]:
housing10['pop2010'] = housing10['pop2010'].astype(int)
housing10['hu2010'] = housing10['hu2010'].astype(int)

housing10.dtypes

pop2010     int64
hu2010      int64
state      object
county     object
STCO       object
dtype: object

In [31]:
hpCSA00_17 = housing00.merge(housing10, on='STCO')
hpCSA00_17 = hpCSA00_17.merge(BPcounties, on='STCO').groupby(['CSA', 'CSA_name']).sum()

print(hpCSA00_17.shape)
hpCSA00_17

(15, 8)


,,pop2000,hu2000,pop2010,hu2010,1-2units_00,+3units_00,1-2units,+3units
CSA,CSA_name,,,,,,,,
122,"Atlanta--Athens-Clarke County--Sandy Springs, GA",4778990,1853158,5910296,2426656,474506,132773,136174,63364
148,"Boston-Worcester-Providence, MA-RI-NH-CT",7630016,3135867,7893376,3380251,162346,62540,71045,57752
176,"Chicago-Naperville, IL-IN-WI",4088612,1516964,4646254,1779354,235555,32241,45693,14690
206,"Dallas-Fort Worth, TX-OK",5565005,2180002,6817483,2708233,362296,116678,189949,150573
216,"Denver-Aurora, CO",2629980,1068646,3034985,1279543,162073,62325,76848,65132
220,"Detroit-Warren-Ann Arbor, MI",5456428,2208124,5318744,2332713,145283,29291,45326,12381
288,"Houston-The Woodlands, TX",4878216,1865805,6114562,2379231,382359,122838,258944,105905
348,"Los Angeles-Long Beach, CA",12365627,4240393,12828837,4493983,132468,131199,62747,128721
370,"Miami-Fort Lauderdale-Port St. Lucie, FL",5475847,2379888,6166766,2774432,232938,148598,60513,78889


### Exporting CSAs housing permits data to .csv

In [32]:
hpCSA00_17.to_csv('exports/BP00-17CSA.csv')

-----

# PLACES 
Major Cities within the Regions

### _Note#1: PLACES in the housing permits survey are separated to Midwest, Northeast, South and West Regions; data were downloaded through each of these for years 2010 - 2017 and were concatenated, then cleaned to include our targeted cities only_

-----

# _TDC on Aug 14th, 2018_
# _** Changes in GEOID's for several places_

### _Note#2: NYC is considered as 5 places, which are its 5 boroughs_

In [71]:
NE = ['ne2017a', 'ne2016a', 'ne2015a', 'ne2014a',
      'ne2013a', 'ne2012a', 'ne2011a', 'ne2010a']
NEdata = []

for year in NE:
    df = 'NE'+year
    df = pd.read_table('https://www2.census.gov/econ/bps/Place/Northeast%20Region/'+
                         year+'.txt', 
                         header=0, sep=r'\,|\t', engine='python').iloc[:,:28]

    df.columns = df.iloc[0]
    df = df[1:].set_index(['Name'])

    df = df.drop(['Bldgs', 'Value'], axis=1)
    df.columns = ['State', '6-Digit', 'County', 'Census Place',
                  'Place','FIPS MCD', 'Pop', 'CSA', 'CBSA',
                  'Footnote', 'Central', 'Zip','Region', 'Division', 
                  'Number of','1unit', '2unit', '3-4unit', '+5unit']
    df = df.drop(['Central', 'Pop', 'Footnote', 'Census Place',
                  '6-Digit', 'FIPS MCD', 'Number of',
                  'Zip', 'Region', 'Division'], axis=1)

    df['1unit'] = df['1unit'].astype(int)
    df['2unit'] = df['2unit'].astype(int)
    df['3-4unit'] = df['3-4unit'].astype(int)
    df['+5unit'] = df['+5unit'].astype(int)

    df['1-2units'] = df['1unit'] + df['2unit']
    df['+3units'] = df['3-4unit'] + df['+5unit']

    # creating 'GEOID' column from state and place
    df['State'] = df['State'].apply(lambda x: '{0:0>2}'.format(x))
    df['Place'] = df['Place'].apply(lambda x: '{0:0>5}'.format(x))

    df['GEOID'] = df[['State', 'Place']].apply(lambda x: ''.join(x), axis=1)

    # Dropping columns
    df = df.drop(['1unit', '2unit', '3-4unit', '+5unit',
                                  'State', 'Place'], axis=1)

    # ONLY FOR NE FILES! >> aggregating NYC 5 boroughs into 1 row: NYC
#     df = df.append(df.loc[['Queens borough', 'Bronx borough', 
#                     'Manhattan borough','Brooklyn borough',
#                     'Staten Island borough']].sum(numeric_only=True),
#                      ignore_index=True)
# #     df['GEOID'] = df['GEOID'].fillna('3651000')
# #     df['CSA'] = df['CSA'].fillna('408')

#     df['1-2units'] = df['1-2units'].astype(int)
#     df['+3units'] = df['+3units'].astype(int)
    df['GEOID'] = df['GEOID'].replace(' ', '')
    for index, row in df.iterrows():
        if row['GEOID'] == '2500025':
            row['GEOID'] = '2507000'
    NEdata.append(df)

NEall = pd.concat(NEdata).groupby(['GEOID']).sum().reset_index()
print(NEall.shape)
print(NEall.dtypes)
NEall.head()

(1949, 3)
GEOID       object
1-2units     int64
+3units      int64
dtype: object


,GEOID,1-2units,+3units
0,09000,8463,2641
1,0900000,8590,5637
2,0901150,20,0
3,0908000,143,628
4,0908420,230,118


In [53]:
# NE2017
placesNE17 = pd.read_table('https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2017a.txt', 
             header=0, sep=r'\,|\t', engine='python').iloc[:,:28]

placesNE17.columns = placesNE17.iloc[0]
placesNE17 = placesNE17[1:].set_index(['Name'])

placesNE17 = placesNE17.drop(['Bldgs', 'Value'], axis=1)
placesNE17.columns = ['State', '6-Digit', 'County', 'Census Place',
                      'Place','FIPS MCD', 'Pop', 'CSA', 'CBSA',
                      'Footnote', 'Central', 'Zip','Region', 'Division', 
                      'Number of','1unit', '2unit', '3-4unit', '+5unit']
placesNE17 = placesNE17.drop(['Central', 'Footnote', 'Census Place',
                              '6-Digit', 'FIPS MCD', 'Number of',
                              'Zip', 'Region', 'Division'], axis=1)

placesNE17['1unit'] = placesNE17['1unit'].astype(int)
placesNE17['2unit'] = placesNE17['2unit'].astype(int)
placesNE17['3-4unit'] = placesNE17['3-4unit'].astype(int)
placesNE17['+5unit'] = placesNE17['+5unit'].astype(int)
placesNE17['Pop'] = placesNE17['Pop'].astype(int)

placesNE17['1-2units'] = placesNE17['1unit'] + placesNE17['2unit']
placesNE17['+3units'] = placesNE17['3-4unit'] + placesNE17['+5unit']

# creating 'GEOID' column from state and place
placesNE17['State'] = placesNE17['State'].apply(lambda x: '{0:0>2}'.format(x))
placesNE17['Place'] = placesNE17['Place'].apply(lambda x: '{0:0>5}'.format(x))

placesNE17['GEOID'] = placesNE17[['State', 'Place']].apply(lambda x: ''.join(x), axis=1)

# Dropping columns
placesNE17 = placesNE17.drop(['1unit', '2unit', '3-4unit', '+5unit',
                              'State', 'Place'], axis=1)

# ONLY FOR NE FILES! >> aggregating NYC 5 boroughs into 1 row: NYC
# placesNE17 = placesNE17.append(placesNE17.loc[['Queens borough', 'Bronx borough', 
#                 'Manhattan borough','Brooklyn borough',
#                 'Staten Island borough']].sum(numeric_only=True),
#                  ignore_index=True)
# placesNE17.set_value(5580, 'CSA', '408')
# placesNE17.set_value(5580, 'GEOID', '3651000')

placesNE17['Pop'] = placesNE17['Pop'].astype(int)
placesNE17['1-2units'] = placesNE17['1-2units'].astype(int)
placesNE17['+3units'] = placesNE17['+3units'].astype(int)


print(placesNE17.shape)
placesNE17.head(2)

(5580, 7)


,County,Pop,CSA,CBSA,1-2units,+3units,GEOID
Name,,,,,,,
Andover town,013,3303,278,25540,3,0,0900000
Ansonia,009,19249,408,35300,6,0,0901150


In [82]:
# NE2016
placesNE16 = pd.read_table('https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2016a.txt', 
            header=0, sep=r'\,|\t', engine='python').iloc[:,:28]

placesNE16.columns = placesNE16.iloc[0]
placesNE16 = placesNE16[1:].set_index(['Name'])

placesNE16 = placesNE16.drop(['Bldgs', 'Value'], axis=1)
placesNE16.columns = ['State', '6-Digit', 'County', 'Census Place',
                      'Place','FIPS MCD', 'Pop', 'CSA', 'CBSA',
                      'Footnote', 'Central', 'Zip','Region', 'Division', 
                      'Number of','1unit', '2unit', '3-4unit', '+5unit']
placesNE16 = placesNE16.drop(['Central', 'Footnote', 'Census Place',
                              '6-Digit', 'FIPS MCD', 'Number of',
                              'Zip', 'Region', 'Division'], axis=1)

placesNE16['1unit'] = placesNE16['1unit'].astype(int)
placesNE16['2unit'] = placesNE16['2unit'].astype(int)
placesNE16['3-4unit'] = placesNE16['3-4unit'].astype(int)
placesNE16['+5unit'] = placesNE16['+5unit'].astype(int)
placesNE16['Pop'] = placesNE16['Pop'].astype(int)

placesNE16['1-2units'] = placesNE16['1unit'] + placesNE16['2unit']
placesNE16['+3units'] = placesNE16['3-4unit'] + placesNE16['+5unit']
placesNE16 = placesNE16.drop(['1unit', '2unit', '3-4unit', '+5unit'], axis=1)

placesNE16['State'] = placesNE16['State'].apply(lambda x: '{0:0>2}'.format(x))
placesNE16['Place'] = placesNE16['County'].apply(lambda x: '{0:0>5}'.format(x))

placesNE16['GEOID'] = placesNE16[['State', 'Place']].apply(lambda x: ''.join(x), axis=1)

# aggregating NYC 5 boroughs into 1 row: NYC
NYC = [placesNE16.loc[['Queens borough', 'Bronx borough', 
                'Manhattan borough','Brooklyn borough',
                'Staten Island borough']]]
placesNE16 = placesNE16.append(NYC).sum(numeric_only=True)
# placesNE17.index[-1:] = placesNE17.index[-1:].rename("New York",inplace=False)

# 3651000
print(placesNE16.shape)
placesNE16.tail(2)

(5578, 9)


,State,County,Place,Pop,CSA,CBSA,1-2units,+3units,GEOID
Name,,,,,,,,,
Woodstock town,50,027,00027,2148,999,99999,6,0,5000027
Woodstock village,50,027,00027,900,999,99999,2,0,5000027


In [86]:
placesNE16.loc[['Queens borough', 'Bronx borough', 
                'Manhattan borough','Brooklyn borough',
                'Staten Island borough']].sum(numeric_only=True)

Pop         8175133
1-2units       1513
+3units       14767
dtype: int64

In [54]:
# NE2015
placesNE15 = pd.read_table('https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2015a.txt', 
            header=0, sep=r'\,|\t', engine='python').iloc[:,:28]

placesNE15.columns = placesNE15.iloc[0]
placesNE15 = placesNE15[1:].set_index(['Name'])

placesNE15 = placesNE15.drop(['Bldgs', 'Value'], axis=1)
placesNE15.columns = ['State', '6-Digit', 'County', 'Census Place',
                      'Place','FIPS MCD', 'Pop', 'CSA', 'CBSA',
                      'Footnote', 'Central', 'Zip','Region', 'Division', 
                      'Number of','1unit', '2unit', '3-4unit', '+5unit']
placesNE15 = placesNE15.drop(['Central', 'Footnote', 'Census Place',
                              '6-Digit', 'FIPS MCD', 'Number of',
                              'Zip', 'Region', 'Division'], axis=1)

placesNE15['1unit'] = placesNE15['1unit'].astype(int)
placesNE15['2unit'] = placesNE15['2unit'].astype(int)
placesNE15['3-4unit'] = placesNE15['3-4unit'].astype(int)
placesNE15['+5unit'] = placesNE15['+5unit'].astype(int)
placesNE15['Pop'] = placesNE15['Pop'].astype(int)

placesNE15['1-2units'] = placesNE15['1unit'] + placesNE15['2unit']
placesNE15['+3units'] = placesNE15['3-4unit'] + placesNE15['+5unit']
placesNE15 = placesNE15.drop(['1unit', '2unit', '3-4unit', '+5unit'], axis=1)

placesNE15['State'] = placesNE15['State'].apply(lambda x: '{0:0>2}'.format(x))
placesNE15['Place'] = placesNE15['County'].apply(lambda x: '{0:0>5}'.format(x))

placesNE15['GEOID'] = placesNE15[['State', 'Place']].apply(lambda x: ''.join(x), axis=1)

# aggregating NYC 5 boroughs into 1 row: NYC
# placesNE17.append(placesNE17.loc[['Queens borough', 'Bronx borough', 
#                 'Manhattan borough','Brooklyn borough',
#                 'Staten Island borough']].sum(numeric_only=True),
#                  ignore_index=True)
# placesNE17.index[-1:] = placesNE17.index[-1:].rename("New York",inplace=False)

# 3651000
print(placesNE15.shape)
placesNE15.head(2)

(5581, 9)


,State,County,Place,Pop,CSA,CBSA,1-2units,+3units,GEOID
Name,,,,,,,,,
Andover town,09,013,00013,3303,278,25540,4,0,0900013
Ansonia,09,009,00009,19249,408,35300,0,0,0900009


In [74]:
NEall[NEall['GEOID'] == ]

0        09000  
1       0900000 
2       0901150 
3       0908000 
4       0908420 
5       0918430 
6       0919480 
7       0927810 
8       0934180 
9       0934460 
10      0937000 
11      0946450 
12      0947290 
13      0949880 
14      0950370 
15      0952000 
16      0952280 
17      0955990 
18      0956200 
19      0968100 
20      0973000 
21      0973700 
22      0976500 
23      0980000 
24      0982800 
25       23000  
26      2300000 
27      2302060 
28      2302100 
29      2302795 
          ...   
1919    4400000 
1920    4414140 
1921    4419180 
1922    4422960 
1923    4449960 
1924    4454640 
1925    4459000 
1926    4474300 
1927    4480780 
1928     50000  
1929    5000000 
1930    5003175 
1931    5010675 
1932    5011050 
1933    5024025 
1934    5024400 
1935    5034975 
1936    5042700 
1937    5046000 
1938    5048850 
1939    5049075 
1940    5050200 
1941    5053125 
1942    5061225 
1943    5061675 
1944    5066175 
1945    5074650 
1946    507690

### Reading in my Geocoded places table
Created by Dara Goldberg

In [27]:
cities = pd.read_excel('data/CSA Population+Change_2010-2017.xlsx', 
             sheet_name='Cities_pop+geoinfo').iloc[:,:5]

# setting GEOID to 7 digits to assure match
cities['GEOID'] = cities['GEOID'].apply(lambda x: '{0:0>7}'.format(x))
# setting GEOID to str
cities.GEOID = cities.GEOID.astype(str)

print(cities.shape)
cities

(19, 5)


,GEOID,NAMELSAD,NAME,CSA,ALAND_mi
0,0644000,"Los Angeles city, California",Los Angeles,348,468.65867
1,0653000,"Oakland city, California",Oakland,488,55.89604
2,0667000,"San Francisco city, California",San Francisco,488,46.90564
3,0668000,"San Jose city, California",San Jose,488,177.51410
4,0820000,"Denver city, Colorado",Denver,216,153.30483
5,1150000,"Washington city, District of Columbia",Washington,548,61.13988
6,1245000,"Miami city, Florida",Miami,370,35.98691
7,1304000,"Atlanta city, Georgia",Atlanta,122,133.43344
8,1714000,"Chicago city, Illinois",Chicago,176,227.34010
9,2507000,"Boston city, Massachusetts",Boston,148,48.34364


In [73]:
cities.merge(NEall, on='GEOID')

,GEOID,NAMELSAD,NAME,CSA,ALAND_mi,1-2units,+3units


In [45]:
NEall.columns

Index(['1-2units', '+3units'], dtype='object')